# Building Deep Network using Grid Search


## Import necessary libraries and functions


In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from time import time
from keras import regularizers
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

# to suppress the warnings
import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Setting random variables to get the same results on different devices
seed = 7
tf.random.set_seed(seed)
np.random.seed(1)
random.seed(1)

### Define additional functions

In [3]:
def plot_history(history, main_title=None):
    """
    Plot training & validation accuracy from history.
    """
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')

    plt.tight_layout()
    if main_title:
        plt.suptitle(main_title, fontsize=16, y=1.05)
    plt.show()

def plot_confusion_matrix(c_matrix, labels):
    """
    Print confusion matrix with sns.heatmap
    """
    ax = sns.heatmap(c_matrix, annot=True, linewidth=1.5, fmt='d',
                     xticklabels=labels,
                     yticklabels=labels,
                     cbar=False, cmap='Blues')

    ax.set_xlabel("Prediction")
    ax.set_ylabel("Actual")
    plt.show()

    
def plot_confusion_matrix_alt(c_matrix, labels):
    """
    sns.heatmap didn't show all values, so this is malual heatmap,
    that works properly.
    """
    
    fig, ax = plt.subplots()
    cax = ax.matshow(c_matrix, cmap='Blues')
    # Add annotations
    for i in range(len(labels)):
        for j in range(len(labels)):
            if i==j:
                color='white'
            else:
                color='black'
            ax.text(j, i, c_matrix[i, j], va='center', ha='center', color=color, fontsize=12)
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    plt.xlabel("Prediction")
    plt.ylabel("Actual")
    plt.show()

## Data loading

In [4]:
# Load a dataset
df = pd.read_csv('/Users/jts/Documents/college/Programming/CA2/glass_data.csv')
df = shuffle(df, random_state=seed)
df.head()

,id,ri,na,mg,al,si,k,ca,ba,fe,type
138,139,1.51674,12.79,3.52,1.54,73.36,0.66,7.90,0.0,0.0,2
65,66,1.52099,13.69,3.59,1.12,71.96,0.09,9.40,0.0,0.0,1
95,96,1.51860,13.36,3.43,1.43,72.26,0.51,8.60,0.0,0.0,2
22,23,1.51736,12.78,3.62,1.29,72.79,0.59,8.70,0.0,0.0,1
63,64,1.52227,14.17,3.81,0.78,71.35,0.00,9.69,0.0,0.0,1


In [6]:
# Define features and target variable
X_features = df.drop('type', axis=1)
y_tagret = df['type']
print('Data shapes:', X_features.shape, y_tagret.shape)

Data shapes: (214, 10) (214,)


## Testing the model architectures and hyperparameters
The hyperparameters that we're going to test:
- units in Dense layer,
- optimizers,
- batch size,
- split size.

In [7]:
def create_model(hidden_units=64, optimizer='adam', input_dim=9):
    # Create basic architecture
    model = Sequential([
        # Input layer
        Dense(hidden_units, input_dim=input_dim, activation='relu'),
        #Hidden layer
        Dense(hidden_units, activation='relu'),
        # Output layer
        Dense(1, activation='sigmoid'),
    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


def perform_grid_search(X, y, param_grid, export_csv: bool =True):
    """
    
    :param X: Independent variables.
    :param y: Dependent variable.
    :param param_grid: Parameters to test.
    :param export_csv: Save full report of each Default is True.
    
    :return  results: A list of dictionaries that includes best models found by GridSearchCV
    """
    # Fix randomness in results
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    # To store full results and create CSV afterwords
    main_df = pd.DataFrame()
    # Create new variable to iterate over and delete it for param_grid dict
    test_sizes = param_grid['test_size']
    del param_grid['test_size']
    
    results = []
    for test_size in test_sizes:
        # Create a KerasClassifier
        model = KerasClassifier(build_fn=create_model, verbose=0)
        # Create a pipeline with a StandardScaler and our model
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model) ])
        # Set Early stopping so model won't overfit
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        
        print('* Start testing of', test_size, 'test size. *')
        # Split your data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


        # Use GridSearchCV to search through the hyperparameter grid
        grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='accuracy', cv=3, verbose=0)
        grid_result = grid.fit(X_train, y_train, model__callbacks=[early_stopping])

        # Store results in a DataFrame
        results_df = pd.DataFrame(grid_result.cv_results_)
        # Add test_size as col to results of iteration
        results_df['test_size'] = [test_size] * len(results_df)

        # Get the best hyperparameters
        best_params = grid_result.best_params_

        # Create a new model with the best hyperparameters
        arch_param = {k.replace('model__', ''): v 
                      for k,v in best_params.items() 
                      if k!='model__batch_size'}
        best_model = create_model(**arch_param)

        # Print the best hyperparameters and corresponding accuracy
        print("Best score: %f using %s" % (grid_result.best_score_, test_size))
        print(best_params)

        # Predict on the test set with the best model
        y_pred = best_model.predict(X_test)

        # Evaluate the accuracy on the test set
        y_pred = np.where(y_pred.flatten().round(2) > 0.5, 1, 0)
        accuracy = accuracy_score(y_test.to_numpy(), y_pred)
        print("Test Accuracy: %.2f%%" % (accuracy * 100))
        pd.concat([main_df, results_df])
        results.append({
            'test_size': test_size,
            'best_param': best_params,
            'accuracy': accuracy,})
        print('* Finished testing', test_size, 'test size. *')
    # Export results to CSV
    if export_csv:
        main_df.to_csv('grid_search_results.csv', index=False)

    return results

In [8]:
param_grid = {
    'model__hidden_units': [4, 16, 512],
    'model__optimizer': [tf.keras.optimizers.legacy.Adam(),
                         tf.keras.optimizers.legacy.SGD(momentum=0.9),
                        # tf.keras.optimizers.legacy.RMSprop()
                        ],
    'model__batch_size': [100, 50, 10],
    'test_size': [.1, .2, .3]
}
# Because GridSearchCV has a pipeline with StandardScaler
# we're passing not scaled X and Y
gs_t1 = time()
# We don't want to show all data, so we take only first 30,000 rows from dataset.
grid_search_results = perform_grid_search(X_features, y_tagret, param_grid)
gs_t2 = time()

* Start testing of 0.1 test size. *


2023-12-26 20:33:33.589954: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: 
All the 54 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jts/anaconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jts/anaconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/jts/anaconda3/envs/ml_env/lib/python3.11/site-packages/keras/wrappers/scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jts/anaconda3/envs/ml_env/lib/python3.11/site-packages/keras/wrappers/scikit

In [ ]:
print(f'Finished in {round((gs_t2-gs_t1)/60)} min.')
# Best models in each value of test split size
grid_search_results

In [ ]:
# Visualise the results
test_sizes = [entry['test_size'] for entry in grid_search_results]
accuracies = [entry['accuracy']*100 for entry in grid_search_results]

plt.plot(test_sizes, accuracies, marker='.', linestyle='-')
plt.title('GridSearch results:\nTest Size vs Accuracy', y=1.01)
plt.xlabel('Test Size')
plt.ylabel('Accuracy')
plt.ylim(round(min(accuracies))-5, round(max(accuracies))+5)
# Add values of each point 
for x, y in zip(test_sizes, accuracies):
    plt.text(x, y, f'{y:.2f}', ha='left', va='bottom', color='blue')
plt.show()

## Choosing parameters and training the model
Based on received results, let's choose the most accurate model to proceed with our task.

In [ ]:
# Select the most optimal model with max accuracy
best_model_params = max(grid_search_results, key=lambda x: x['accuracy'])
# Print parameters of the most optimal model
print('The most optimal model parameters\n\tTest size:', best_model_params['test_size'])
for k, v in best_model_params['best_param'].items():
    k = k.replace('model__', '').replace('_', " ")
    k = k[0].upper() + k[1:]
    if k == 'Optimizer':
        print(f'\t{k}: {v.__class__.__name__}')
    else:
        print(f'\t{k}: {v}')

### Split data

In [ ]:
# Split the data into training and test sets with found test_size
real_size = round(X_features.shape[0]/30000)*best_model_params['test_size']
X_train, X_test, y_train, y_test = train_test_split(X_features, y_tagret, 
                                                    test_size=real_size,
                                                    random_state=seed)

# Scaling of features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

### Define the learning rate

In [ ]:
def tune_lr(X_train, y_train, epochs=50):
    model_tune = Sequential()
    model_tune.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model_tune.add(Dense(64, activation='relu'))
    model_tune.add(Dense(1, activation='sigmoid'))
    
    lr_schedule = LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))
    
    optimizer = tf.keras.optimizers.legacy.SGD(momentum=0.9) # for M2 processor only
    # optimizer = tf.keras.optimizers.Adam()  
    
    model_tune.compile(loss='binary_crossentropy', optimizer=optimizer)

    history = model_tune.fit(X_train, y_train, epochs=epochs, callbacks=[lr_schedule], verbose=0)
    lrs = 1e-8 * (10 ** (np.arange(epochs) / 20))
    # Plot the learning rate graph
    plt.figure(figsize=(10, 6))
    plt.grid(True)
    plt.semilogx(lrs, history.history["loss"])
    plt.axis([min(lrs), max(lrs), min(history.history["loss"]), max(history.history["loss"])])
    plt.xlabel('Learning Rate')
    plt.ylabel('binary_crossentropy')
    plt.tick_params('both', length=10, width=1, which='both')
    plt.show()

    optimal_lr_index = np.argmin(history.history["loss"])
    optimal_lr = lrs[optimal_lr_index]

    print(f'Optimal Learning Rate: {optimal_lr}')
    return optimal_lr

In [ ]:
# Find learning rate
optimal_lr = tune_lr(X_train, y_train, epochs=50)

### Create the model
Build, train and evaluate the model with chosen hyperparameters.

In [ ]:
hidden_units = best_model_params['best_param']['model__hidden_units']
batch_size = best_model_params['best_param']['model__batch_size']
# Get the class of the best optimizer
optimizer_class = best_model_params['best_param']['model__optimizer'].__class__
# Create a new optimizer instance with the same configuration
optimizer = optimizer_class()
optimizer.learning_rate.assign(optimal_lr)

# We are increasing no of epochs,
# because we added learning_rate to optimizer
epochs = 500

In [ ]:
# Build the model
model = Sequential([
        # Input layer
        Dense(hidden_units, input_dim=X_train.shape[1], activation='relu'),
        #Hidden layer
        Dense(hidden_units, activation='relu'),
        # Output layer
        Dense(1, activation='sigmoid'),
    ])
# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

### Train the model

In [ ]:
# Declare own callback class to stop training when conditions met 
class val_metrics_stop(tf.keras.callbacks.Callback):
    def __init__(self, patience=10):
        self.patience = patience
    def on_epoch_end(self, epoch, logs={}):
        """
        To stop the training when the val_loss or val_accuracy meets conditions.
        """
        self.stopped_epoch = 0
        if self.patience < epoch and (logs.get('val_loss') < 0.01 or logs.get('val_accuracy') > 0.9999):
                # Stop if threshold is met
                print("\nCancelling training!")
                self.model.stop_training = True
                self.stopped_epoch = epoch

In [ ]:
# EarlyStopping will stop training
# when 'val_loss' is the same for 5 epochs
val_loss_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
val_accuracy_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)
# Instantiate own callback class
val_metrics_stop_numbers = val_metrics_stop(patience=10)

tm1 = time()
model_history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=epochs, batch_size=batch_size,
                    callbacks=[val_metrics_stop_numbers, val_loss_stop, val_accuracy_stop],)
tm2 = time()

### Evaluation of training

In [ ]:
# Plot learning curves
plot_history(model_history)

In [ ]:
# Calculating and plotting confusion_matrix
y_test_pred = model.predict(X_test, verbose=0)
y_test_pred_new = np.where(y_test_pred.flatten().round(2) > 0.5, 1, 0)
c_matrix = confusion_matrix(y_test, y_test_pred_new)
labels = ['Not CAFV', 'CAFV']

plot_confusion_matrix(c_matrix, labels)
plot_confusion_matrix_alt(c_matrix, labels)

In [ ]:
# No of epochs
epochs_list = list({val_metrics_stop_numbers.stopped_epoch, val_accuracy_stop.stopped_epoch, val_loss_stop.stopped_epoch, epochs})
epochs_list.sort()
epochs_trained = min(epochs_list[1:])
print('Trained on', epochs_trained, 'epochs')

y_pred = model.predict(X_test, verbose=0)
y_pred = np.where(y_pred.flatten().round(2) > 0.5, 1, 0)
# Calculate evaluation metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the metrics
scores = model.evaluate(X_train, y_train, verbose=0,)
print(f"Training Accuracy: {round(scores[1]*100, 4)}%")
scores = model.evaluate(X_test, y_test, verbose=0,)
tes_a_1 = scores[1]*100
print(f"Testing Accuracy: {round(scores[1]*100, 4)}%")
print(f"Precision: {precision:.4f}") # fraction of positive predictions which are correct
print(f"Recall: {recall:.4f}") # True positive rate 
print('Time spent on training:', round(tm2-tm1), 'seconds.')